In [ ]:
##This script was used with the callback "CSV logger" to obtain the excel file which was used for following the evolution of loss function and binary accuracy with all the optimisers
##It also uses a Learning Rate Scheduler which helps us set from the start which lr we use in each iteration without having to save and load the weights

In [27]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\callbacks

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\callbacks


In [28]:
import numpy as np

In [29]:
def convertStringIntoVector (String):##Receives a binary vector in the form of a String into a numpy array
    intoList=list(String)
    vector=np.asarray(intoList,dtype=int)
    return vector

In [30]:
def splittingTrainingDataIntoXAndY (filesData):##Receives a list in which element has as [0] the X and as [1] the Y
    xData=[]
    yData=[]
    for element in filesData:
        xData.append(element[0])
        yData.append(element[1])
    return xData, yData

In [31]:
filesData=[]
numberOfClusters=0
XFile=open("pfamDomains0and1ForAllClusters.txt")
YFile=open("fingerPrintsForIndividualClustersNOT478.txt")
for pfam, fingerprint in zip(XFile,YFile): ##Reads the two files at the same time and adds each thing to the same list
    pfam=pfam.strip("\n")
    fingerprint=fingerprint.strip("\n")
    filesData.append([convertStringIntoVector(pfam[11:]),convertStringIntoVector(fingerprint[11:])])
    numberOfClusters+=1
XFile.close()
YFile.close()

In [32]:
def gaussianKernel (vector1, vector2):##Does the gaussian Kernel for two vectors
    gamma=0.1
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [33]:
xData,yData = splittingTrainingDataIntoXAndY(filesData)##Now the list is splitted into X and Y data
xData=np.concatenate(xData,axis=0)
xData=xData.reshape(numberOfClusters,6159)
yData=np.concatenate(yData,axis=0)
yData=yData.reshape(numberOfClusters,306)
#The reshape puts the data as one row for each training example

In [34]:
import random
#Creates a list of Random Numbers as big as the number of Clusters we have
#Then we will select which ones we have for each subset
listOfRandomNumbers=random.sample(range(numberOfClusters), numberOfClusters)

In [35]:
xTestData = np.asarray([xData[i] for i in listOfRandomNumbers[:206]],dtype=int)
xTrainingData = np.asarray([xData[i] for i in listOfRandomNumbers[206:]],dtype=int)
yTestData = np.asarray([yData[i] for i in listOfRandomNumbers[:206]],dtype=int)
yTrainingData = np.asarray([yData[i]for i in listOfRandomNumbers[206:]],dtype=int)

In [36]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from keras import initializers
from keras.callbacks import LearningRateScheduler, CSVLogger, EarlyStopping, TensorBoard
lr=0.01
nn=[6159, 50, 306]

In [37]:
## ADAM

In [38]:
#### CALL BACKS
def learningRateScheduler (epoch):
    if epoch <25:
        return 0.01
    elif epoch<60:
        return 0.003
    else:
        return 0.001
callback1 = LearningRateScheduler(learningRateScheduler,verbose=0)
callback2 = CSVLogger("adamCSVLogger.csv", separator=',', append=False)
callback3 = EarlyStopping(monitor='binary_crossentropy', min_delta=0.001, patience=3, verbose=1, mode='min')

In [39]:
model=Sequential()
model.add(Dense(nn[0], input_dim=6159, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.adam(lr=lr), metrics = ['binary_accuracy'])
model.fit(xTrainingData, yTrainingData,callbacks=[callback1,callback2,callback3], epochs=80)

Epoch 1/80
825/825 [==============================] - 18s 22ms/step - loss: 0.2305 - binary_accuracy: 0.9042
Epoch 2/80
825/825 [==============================] - 15s 18ms/step - loss: 0.1232 - binary_accuracy: 0.9523
Epoch 3/80
825/825 [==============================] - 15s 18ms/step - loss: 0.1114 - binary_accuracy: 0.9553
Epoch 4/80
825/825 [==============================] - 15s 18ms/step - loss: 0.1021 - binary_accuracy: 0.9594
Epoch 5/80
825/825 [==============================] - 15s 18ms/step - loss: 0.0937 - binary_accuracy: 0.9622
Epoch 6/80
825/825 [==============================] - 15s 18ms/step - loss: 0.0897 - binary_accuracy: 0.9639
Epoch 7/80
825/825 [==============================] - 15s 19ms/step - loss: 0.0852 - binary_accuracy: 0.9657
Epoch 8/80
825/825 [==============================] - 15s 19ms/step - loss: 0.0825 - binary_accuracy: 0.9666
Epoch 9/80
825/825 [==============================] - 15s 18ms/step - loss: 0.0798 - binary_accuracy: 0.9677
Epoch 10/80
825/825

In [40]:
predictions = model.predict(xTestData) ##Rounds up and down every prediction to 1 and 0
for i in range (0, len (predictions)):
    for j in range (0, len (predictions[0])):
        if predictions[i][j]<0.5:
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1

In [41]:
score1 = np.zeros((206, 206))##Creates the ouputfile ready to be opened with analysing Scores notebook
for i in range (0, 206):
    for j in range (0, 206):
        score1[i,j]= gaussianKernel(predictions[i],yTestData[j])
np.save("kerasScore1",score1)

In [42]:
## SGD

In [43]:
#### CALL BACKS
def learningRateScheduler (epoch):
    if epoch <25:
        return 0.01
    elif epoch<60:
        return 0.003
    else:
        return 0.001
callback1 = LearningRateScheduler(learningRateScheduler,verbose=0)
callback2 = CSVLogger("sGDCSVLogger.csv", separator=',', append=False)
callback3 = EarlyStopping(monitor='binary_crossentropy', min_delta=0.001, patience=3, verbose=1, mode='min')

In [44]:
model=Sequential()
model.add(Dense(nn[0], input_dim=6159, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=lr), metrics = ['binary_accuracy'])
model.fit(xTrainingData, yTrainingData,callbacks=[callback1,callback2,callback3], epochs=80)

Epoch 1/80
825/825 [==============================] - 10s 12ms/step - loss: 0.6949 - binary_accuracy: 0.5086
Epoch 2/80
825/825 [==============================] - 8s 10ms/step - loss: 0.6938 - binary_accuracy: 0.5148
Epoch 3/80
825/825 [==============================] - 8s 10ms/step - loss: 0.6928 - binary_accuracy: 0.5209
Epoch 4/80
825/825 [==============================] - 8s 10ms/step - loss: 0.6918 - binary_accuracy: 0.5271
Epoch 5/80
825/825 [==============================] - 8s 10ms/step - loss: 0.6909 - binary_accuracy: 0.5333
Epoch 6/80
825/825 [==============================] - 8s 10ms/step - loss: 0.6899 - binary_accuracy: 0.5395
Epoch 7/80
825/825 [==============================] - 8s 10ms/step - loss: 0.6890 - binary_accuracy: 0.5452
Epoch 8/80
825/825 [==============================] - 8s 10ms/step - loss: 0.6880 - binary_accuracy: 0.5511
Epoch 9/80
825/825 [==============================] - 8s 10ms/step - loss: 0.6870 - binary_accuracy: 0.5563
Epoch 10/80
825/825 [======

In [45]:
predictions = model.predict(xTestData) ##Rounds up and down every prediction to 1 and 0
for i in range (0, len (predictions)):
    for j in range (0, len (predictions[0])):
        if predictions[i][j]<0.5:
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1

In [46]:
score2 = np.zeros((206, 206))##Creates the ouputfile ready to be opened with analysing Scores notebook
for i in range (0, 206):
    for j in range (0, 206):
        score2[i,j]= gaussianKernel(predictions[i],yTestData[j])
np.save("kerasScore2",score2)

In [47]:
## RMSrop

In [48]:
#### CALL BACKS
def learningRateScheduler (epoch):
    if epoch <25:
        return 0.01
    elif epoch<60:
        return 0.003
    else:
        return 0.001
callback1 = LearningRateScheduler(learningRateScheduler,verbose=0)
callback2 = CSVLogger("rMSpropCSVLogger.csv", separator=',', append=False)
callback3 = EarlyStopping(monitor='binary_crossentropy', min_delta=0.001, patience=3, verbose=1, mode='min')

In [49]:
model=Sequential()
model.add(Dense(nn[0], input_dim=6159, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=lr), metrics = ['binary_accuracy'])
model.fit(xTrainingData, yTrainingData,callbacks=[callback1,callback2,callback3], epochs=80)

Epoch 1/80
825/825 [==============================] - 14s 17ms/step - loss: 1.1493 - binary_accuracy: 0.9099
Epoch 2/80
825/825 [==============================] - 12s 15ms/step - loss: 1.1594 - binary_accuracy: 0.9277
Epoch 3/80
825/825 [==============================] - 12s 15ms/step - loss: 1.1594 - binary_accuracy: 0.9277
Epoch 4/80
825/825 [==============================] - 12s 15ms/step - loss: 1.1834 - binary_accuracy: 0.9262
Epoch 5/80
825/825 [==============================] - 12s 15ms/step - loss: 1.1888 - binary_accuracy: 0.9258
Epoch 6/80
825/825 [==============================] - 12s 15ms/step - loss: 1.1888 - binary_accuracy: 0.9258
Epoch 7/80
825/825 [==============================] - 12s 15ms/step - loss: 1.1888 - binary_accuracy: 0.9258
Epoch 8/80
825/825 [==============================] - 12s 15ms/step - loss: 1.1888 - binary_accuracy: 0.9258
Epoch 9/80
825/825 [==============================] - 12s 15ms/step - loss: 1.1888 - binary_accuracy: 0.9258
Epoch 10/80
825/825

In [50]:
predictions = model.predict(xTestData) ##Rounds up and down every prediction to 1 and 0
for i in range (0, len (predictions)):
    for j in range (0, len (predictions[0])):
        if predictions[i][j]<0.5:
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1

In [51]:
score3 = np.zeros((206, 206))##Creates the ouputfile ready to be opened with analysing Scores notebook
for i in range (0, 206):
    for j in range (0, 206):
        score3[i,j]= gaussianKernel(predictions[i],yTestData[j])
np.save("kerasScore3",score3)

In [52]:
## Adagrad

In [53]:
#### CALL BACKS
def learningRateScheduler (epoch):
    if epoch <25:
        return 0.01
    elif epoch<60:
        return 0.003
    else:
        return 0.001
callback1 = LearningRateScheduler(learningRateScheduler,verbose=0)
callback2 = CSVLogger("adagradCSVLogger.csv", separator=',', append=False)
callback3 = EarlyStopping(monitor='binary_crossentropy', min_delta=0.001, patience=3, verbose=1, mode='min')

In [54]:
model=Sequential()
model.add(Dense(nn[0], input_dim=6159, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adagrad(lr=lr), metrics = ['binary_accuracy'])
model.fit(xTrainingData, yTrainingData,callbacks=[callback1,callback2,callback3], epochs=80)

Epoch 1/80
825/825 [==============================] - 13s 16ms/step - loss: 0.2359 - binary_accuracy: 0.9042
Epoch 2/80
825/825 [==============================] - 11s 13ms/step - loss: 0.1205 - binary_accuracy: 0.9520
Epoch 3/80
825/825 [==============================] - 11s 13ms/step - loss: 0.1136 - binary_accuracy: 0.9542
Epoch 4/80
825/825 [==============================] - 11s 13ms/step - loss: 0.1086 - binary_accuracy: 0.9558
Epoch 5/80
825/825 [==============================] - 11s 13ms/step - loss: 0.1037 - binary_accuracy: 0.9585
Epoch 6/80
825/825 [==============================] - 11s 13ms/step - loss: 0.0973 - binary_accuracy: 0.9613
Epoch 7/80
825/825 [==============================] - 11s 13ms/step - loss: 0.0924 - binary_accuracy: 0.9631
Epoch 8/80
825/825 [==============================] - 11s 13ms/step - loss: 0.0879 - binary_accuracy: 0.9652
Epoch 9/80
825/825 [==============================] - 11s 13ms/step - loss: 0.0841 - binary_accuracy: 0.9668
Epoch 10/80
825/825

In [55]:
predictions = model.predict(xTestData) ##Rounds up and down every prediction to 1 and 0
for i in range (0, len (predictions)):
    for j in range (0, len (predictions[0])):
        if predictions[i][j]<0.5:
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1

In [56]:
score4 = np.zeros((206, 206))##Creates the ouputfile ready to be opened with analysing Scores notebook
for i in range (0, 206):
    for j in range (0, 206):
        score4[i,j]= gaussianKernel(predictions[i],yTestData[j])
np.save("kerasScore4",score4)

In [57]:
## Adadelta

In [58]:
#### CALL BACKS
def learningRateScheduler (epoch):
    if epoch <25:
        return 0.01
    elif epoch<60:
        return 0.003
    else:
        return 0.001
callback1 = LearningRateScheduler(learningRateScheduler,verbose=0)
callback2 = CSVLogger("adadeltaCSVLogger.csv", separator=',', append=False)
callback3 = EarlyStopping(monitor='binary_crossentropy', min_delta=0.001, patience=3, verbose=1, mode='min')

In [59]:
model=Sequential()
model.add(Dense(nn[0], input_dim=6159, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adadelta(lr=lr), metrics = ['binary_accuracy'])
model.fit(xTrainingData, yTrainingData,callbacks=[callback1,callback2,callback3], epochs=80)

Epoch 1/80
825/825 [==============================] - 21s 26ms/step - loss: 0.6971 - binary_accuracy: 0.5265
Epoch 2/80
825/825 [==============================] - 18s 22ms/step - loss: 0.6953 - binary_accuracy: 0.5319
Epoch 3/80
825/825 [==============================] - 18s 22ms/step - loss: 0.6937 - binary_accuracy: 0.5369
Epoch 4/80
825/825 [==============================] - 18s 22ms/step - loss: 0.6921 - binary_accuracy: 0.5421
Epoch 5/80
825/825 [==============================] - 18s 22ms/step - loss: 0.6906 - binary_accuracy: 0.5472
Epoch 6/80
825/825 [==============================] - 18s 22ms/step - loss: 0.6891 - binary_accuracy: 0.5524
Epoch 7/80
825/825 [==============================] - 18s 22ms/step - loss: 0.6876 - binary_accuracy: 0.5575
Epoch 8/80
825/825 [==============================] - 18s 22ms/step - loss: 0.6862 - binary_accuracy: 0.5628
Epoch 9/80
825/825 [==============================] - 18s 22ms/step - loss: 0.6846 - binary_accuracy: 0.5684
Epoch 10/80
825/825

In [60]:
predictions = model.predict(xTestData) ##Rounds up and down every prediction to 1 and 0
for i in range (0, len (predictions)):
    for j in range (0, len (predictions[0])):
        if predictions[i][j]<0.5:
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1

In [61]:
score5 = np.zeros((206, 206))##Creates the ouputfile ready to be opened with analysing Scores notebook
for i in range (0, 206):
    for j in range (0, 206):
        score5[i,j]= gaussianKernel(predictions[i],yTestData[j])
np.save("kerasScore5",score5)

In [62]:
## Adamax

In [63]:
#### CALL BACKS
def learningRateScheduler (epoch):
    if epoch <25:
        return 0.01
    elif epoch<60:
        return 0.003
    else:
        return 0.001
callback1 = LearningRateScheduler(learningRateScheduler,verbose=0)
callback2 = CSVLogger("adamaxCSVLogger.csv", separator=',', append=False)
callback3 = EarlyStopping(monitor='binary_crossentropy', min_delta=0.001, patience=3, verbose=1, mode='min')

In [64]:
model=Sequential()
model.add(Dense(nn[0], input_dim=6159, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adamax(lr=lr), metrics = ['binary_accuracy'])
model.fit(xTrainingData, yTrainingData,callbacks=[callback1,callback2,callback3], epochs=80)

Epoch 1/80
825/825 [==============================] - 16s 20ms/step - loss: 0.2405 - binary_accuracy: 0.8899
Epoch 2/80
825/825 [==============================] - 14s 16ms/step - loss: 0.1203 - binary_accuracy: 0.9525
Epoch 3/80
825/825 [==============================] - 14s 17ms/step - loss: 0.1099 - binary_accuracy: 0.9559
Epoch 4/80
825/825 [==============================] - 14s 16ms/step - loss: 0.0994 - binary_accuracy: 0.9604
Epoch 5/80
825/825 [==============================] - 14s 16ms/step - loss: 0.0910 - binary_accuracy: 0.9638
Epoch 6/80
825/825 [==============================] - 14s 16ms/step - loss: 0.0841 - binary_accuracy: 0.9669
Epoch 7/80
825/825 [==============================] - 14s 17ms/step - loss: 0.0787 - binary_accuracy: 0.9693
Epoch 8/80
825/825 [==============================] - 14s 16ms/step - loss: 0.0758 - binary_accuracy: 0.9702
Epoch 9/80
825/825 [==============================] - 14s 17ms/step - loss: 0.0719 - binary_accuracy: 0.9716
Epoch 10/80
825/825

In [65]:
predictions = model.predict(xTestData) ##Rounds up and down every prediction to 1 and 0
for i in range (0, len (predictions)):
    for j in range (0, len (predictions[0])):
        if predictions[i][j]<0.5:
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1

In [66]:
score6 = np.zeros((206, 206))##Creates the ouputfile ready to be opened with analysing Scores notebook
for i in range (0, 206):
    for j in range (0, 206):
        score6[i,j]= gaussianKernel(predictions[i],yTestData[j])
np.save("kerasScore6",score6)

In [67]:
## Nadam

In [68]:
#### CALL BACKS
def learningRateScheduler (epoch):
    if epoch <25:
        return 0.01
    elif epoch<60:
        return 0.003
    else:
        return 0.001
callback1 = LearningRateScheduler(learningRateScheduler,verbose=0)
callback2 = CSVLogger("nadamCSVLogger.csv", separator=',', append=False)
callback3 = EarlyStopping(monitor='binary_crossentropy', min_delta=0.001, patience=3, verbose=1, mode='min')

In [69]:
model=Sequential()
model.add(Dense(nn[0], input_dim=6159, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.Nadam(lr=lr), metrics = ['binary_accuracy'])
model.fit(xTrainingData, yTrainingData,callbacks=[callback1,callback2,callback3], epochs=80)

Epoch 1/80
825/825 [==============================] - 21s 25ms/step - loss: 0.2318 - binary_accuracy: 0.9123
Epoch 2/80
825/825 [==============================] - 18s 21ms/step - loss: 0.1202 - binary_accuracy: 0.9531
Epoch 3/80
825/825 [==============================] - 18s 21ms/step - loss: 0.1083 - binary_accuracy: 0.9574
Epoch 4/80
825/825 [==============================] - 18s 21ms/step - loss: 0.0981 - binary_accuracy: 0.9614
Epoch 5/80
825/825 [==============================] - 18s 21ms/step - loss: 0.0915 - binary_accuracy: 0.9634
Epoch 6/80
825/825 [==============================] - 18s 21ms/step - loss: 0.0877 - binary_accuracy: 0.9647
Epoch 7/80
825/825 [==============================] - 18s 21ms/step - loss: 0.0841 - binary_accuracy: 0.9664
Epoch 8/80
825/825 [==============================] - 18s 21ms/step - loss: 0.0817 - binary_accuracy: 0.9673
Epoch 9/80
825/825 [==============================] - 18s 21ms/step - loss: 0.0804 - binary_accuracy: 0.9684
Epoch 10/80
825/825

In [70]:
predictions = model.predict(xTestData) ##Rounds up and down every prediction to 1 and 0
for i in range (0, len (predictions)):
    for j in range (0, len (predictions[0])):
        if predictions[i][j]<0.5:
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1

In [71]:
score7 = np.zeros((206, 206))##Creates the ouputfile ready to be opened with analysing Scores notebook
for i in range (0, 206):
    for j in range (0, 206):
        score7[i,j]= gaussianKernel(predictions[i],yTestData[j])
np.save("kerasScore7",score7)